In [1]:
import sys
import os

parent_path = os.path.abspath("..")  # Get absolute path of parent directory
if parent_path not in sys.path:  
    sys.path.append(parent_path)  # Add only if it's not already there
    
from dotenv import load_dotenv
load_dotenv("../.env")


import pandas as pd
import openai

from src.models.gpt_models import generate_variant_question

### Read Excel file

In [2]:
question_answer_df = pd.read_excel("../data/question_answer/QA_test.xlsx")

print(question_answer_df)

print("Excel Length: ", len(question_answer_df))

   no                    base_question  Để cây tươi tốt  variant_question
0   1   Tại sao cần bón phân cho cây ?              NaN               NaN
1   2               Đạm Cà Mau ở đâu ?              NaN               NaN
2   3  Cây nhãn chăm sóc như thế nào ?              NaN               NaN
Excel Length:  3


### Loop over to get row

In [3]:
import concurrent.futures

# Wrapper function for parallel processing
def process_row(index, row):
    return index, generate_variant_question(row["base_question"])

# Run in parallel with ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:  # Adjust workers as needed
    futures = {executor.submit(process_row, idx, row): idx for idx, row in question_answer_df.iterrows()}
    for future in concurrent.futures.as_completed(futures):
        idx, variant = future.result()
        #print(variant, type(variant))
        question_answer_df["variant_question"] = question_answer_df["variant_question"].astype(str)
        question_answer_df.at[idx, "variant_question"] = str("\n".join(variant.quest_list))

# Save the updated DataFrame to a new Excel file
question_answer_df.to_excel("../data/question_answer/updated_file.xlsx", index=False)